In [1]:
import polars as pl
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from time import perf_counter


In [2]:
orig_x, orig_y = make_classification(n_samples = 100_000, n_features = 10, n_informative = 5, n_redundant = 5)
# This is a Polars dataframe. This is dsds package's favored dataframe. dsds relies on Polars heavily.
# You must turn other dataframe formats into Polars for dsds to work.
df = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y)) 
# Turn it into Pandas.
df_pd = df.to_pandas()
target = "target"
features = df.columns
features.remove(target)

In [3]:
df.head()

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,-1.425278,0.896281,0.65712,-1.287397,-0.140885,-0.304476,-0.875095,-0.726022,0.705475,1.131476
1,2.410247,-0.872861,2.331392,-0.842593,2.490535,-0.216106,-1.024068,-1.384413,2.948725,-1.236277
1,-3.664988,0.63686,-3.076659,0.591886,-0.717415,0.788134,-1.601459,0.948254,-1.223703,2.409829
0,0.735667,-1.961367,-0.442005,1.462097,1.005657,-0.133928,0.042314,-2.020835,-0.308752,0.927343
1,0.795572,-2.283703,-0.79205,1.310753,3.036701,-0.086737,-2.200954,-2.324223,2.238866,1.604933


## Comparisons

This notebook compares results and performance between the dsds package, sklearn and some other packages for feature selection and some other transformations common in the data science pipeline.

### Methods Compared:
1. Scaling and Imputation
2. Fscore
3. Mutual Information Score
4. MRMR feature selection strategies
5. Power Transform

You may restart the kernel after each section. But remember to rerun the cells above. If you are concerned about memory usage when running this notebook, go to the end and run the gc cell.

# Scaling and Imputation

In [4]:
import dsds.transform as t

In [5]:
features = df.columns
features.remove("target")

In [6]:
scaled = t.scale(df, cols=features, strategy="standard")
scaled.head()

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,-0.562045,0.703683,0.594768,-0.796847,-0.723448,-0.686735,0.115639,-0.472,0.145828,0.733403
1,1.360175,-0.36602,1.694854,-0.520007,1.203624,-0.613754,-0.019663,-0.898988,1.746528,-0.801566
1,-1.684503,0.546825,-1.858526,0.372794,-1.14566,0.215598,-0.544064,0.61382,-1.230761,1.562135
0,0.520939,-1.024181,-0.127417,0.914401,0.1162,-0.545888,0.948854,-1.311727,-0.577886,0.601067
1,0.550961,-1.21908,-0.357415,0.820207,1.603598,-0.506916,-1.088541,-1.508483,1.239999,1.040336


In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
# The difference in result is caused by using ddof = 1 for sample variance in dsds
# and using ddof = 0 in sklearn.

# Long and convoluted code just to do some scaling...
std = StandardScaler()
scaled2 = std.fit_transform(df_pd[features], df_pd[target])
# scaled2[:5, :] # scaled2 is a numpy matrix
scaled2 = pd.DataFrame(scaled2, columns=features)
scaled2[target] = df_pd[target]
scaled2.head()

,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,target
0,-0.562048,0.703687,0.594771,-0.796851,-0.723452,-0.686739,0.115639,-0.472003,0.145829,0.733406,0
1,1.360182,-0.366022,1.694863,-0.520010,1.203630,-0.613757,-0.019663,-0.898992,1.746537,-0.801570,1
2,-1.684512,0.546828,-1.858536,0.372796,-1.145665,0.215599,-0.544067,0.613823,-1.230767,1.562143,1
3,0.520941,-1.024186,-0.127418,0.914406,0.116201,-0.545890,0.948858,-1.311733,-0.577889,0.601070,0
4,0.550964,-1.219086,-0.357417,0.820211,1.603606,-0.506918,-1.088546,-1.508490,1.240005,1.040341,1


In [9]:
%%timeit
scaled = t.scale(df, cols=features, strategy="standard")

2.79 ms ± 89.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit
std = StandardScaler()
scaled2 = std.fit_transform(df_pd[features], df_pd[target])
scaled2 = pd.DataFrame(scaled2, columns=features)
scaled2[target] = df_pd[target]


13.4 ms ± 422 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
import dsds.transform as t
t.impute(df, cols=features, strategy="median").head(3) 

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,-1.425278,0.896281,0.65712,-1.287397,-0.140885,-0.304476,-0.875095,-0.726022,0.705475,1.131476
1,2.410247,-0.872861,2.331392,-0.842593,2.490535,-0.216106,-1.024068,-1.384413,2.948725,-1.236277
1,-3.664988,0.63686,-3.076659,0.591886,-0.717415,0.788134,-1.601459,0.948254,-1.223703,2.409829


In [12]:
from sklearn.impute import SimpleImputer 
imputer = SimpleImputer(strategy = "median")
imputed = pd.DataFrame(imputer.fit_transform(df_pd, df_pd[target]), columns=df.columns)
imputed.head(3)

,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
0,0.0,-1.425278,0.896281,0.657120,-1.287397,-0.140885,-0.304476,-0.875095,-0.726022,0.705475,1.131476
1,1.0,2.410247,-0.872861,2.331392,-0.842593,2.490535,-0.216106,-1.024068,-1.384413,2.948725,-1.236277
2,1.0,-3.664988,0.636860,-3.076659,0.591886,-0.717415,0.788134,-1.601459,0.948254,-1.223703,2.409829


In [13]:
%%timeit
t.impute(df, cols=features, strategy="median")

1.31 ms ± 12 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [14]:
%%timeit
imputer = SimpleImputer(strategy = "median")
imputed = pd.DataFrame(imputer.fit_transform(df_pd, df_pd[target]), columns=df.columns)

122 ms ± 537 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Fscore

In [15]:
import dsds.fs as fs # fs = feature_selection
from sklearn.feature_selection import mutual_info_classif, f_classif, f_regression

In [16]:
fs._f_score(df, target=target, num_list = features)

array([4.25789481e+03, 9.84678971e+00, 4.36102858e+02, 1.42781729e+00,
       1.29641800e+03, 5.26671074e+03, 1.06998815e+00, 4.66347309e-01,
       1.38482805e+04, 2.62524671e-01])

In [17]:
# The more core, the bigger the difference. Data here is not big enough to show the difference
start = perf_counter()
res = fs.f_classif(df, target=target)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing Fscore.")
res.head(10)

Spent 0.00s in computing Fscore.


feature,f_value,p_value
str,f64,f64
"""column_0""",4257.894809,0.0
"""column_1""",9.84679,0.001702
"""column_2""",436.102858,1.2288e-96
"""column_3""",1.427817,0.232124
"""column_4""",1296.417998,4.4020e-282
"""column_5""",5266.710735,0.0
"""column_6""",1.069988,0.300951
"""column_7""",0.466347,0.494674
"""column_8""",13848.280461,0.0


In [18]:
start = perf_counter()
f, pv = f_classif(df_pd[features], df_pd[target])
res = pd.DataFrame({"feature":features, "f_value":f, "p_value":pv})
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing Fscore.")
res.head(10)

Spent 0.02s in computing Fscore.


,feature,f_value,p_value
0,column_0,4257.894809,0.000000e+00
1,column_1,9.846790,1.701778e-03
2,column_2,436.102858,1.228752e-96
3,column_3,1.427817,2.321238e-01
4,column_4,1296.417998,4.402006e-282
5,column_5,5266.710735,0.000000e+00
6,column_6,1.069988,3.009506e-01
7,column_7,0.466347,4.946740e-01
8,column_8,13848.280461,0.000000e+00
9,column_9,0.262525,6.083921e-01


In [19]:
start = perf_counter()
f, pv = f_regression(df_pd[features], df_pd[target])
res = pd.DataFrame({"feature":features, "f_value":f, "p_value":pv})
end = perf_counter()
print(f"Spent {end - start: .2f}s in computing Fscore.")
res.head(10)

Spent  0.01s in computing Fscore.


,feature,f_value,p_value
0,column_0,4257.894809,0.000000e+00
1,column_1,9.846790,1.701778e-03
2,column_2,436.102858,1.228752e-96
3,column_3,1.427817,2.321238e-01
4,column_4,1296.417998,4.402006e-282
5,column_5,5266.710735,0.000000e+00
6,column_6,1.069988,3.009506e-01
7,column_7,0.466347,4.946740e-01
8,column_8,13848.280461,0.000000e+00
9,column_9,0.262525,6.083921e-01


# Mutual Information Score

In [20]:
# Vastly faster than sklearn. Finished in 0.7s in this run
fs.mutual_info(df, target=target, conti_cols=features).sort(by="estimated_mi", descending=True).limit(10)

Mutual Info: 100%|██████████| 10/10 [00:00<00:00, 13.15it/s]


feature,estimated_mi
str,f64
"""column_8""",0.086362
"""column_5""",0.045589
"""column_2""",0.04045
"""column_0""",0.036647
"""column_1""",0.03106
"""column_9""",0.022108
"""column_4""",0.011846
"""column_6""",0.003619
"""column_3""",0.000802


In [21]:
def estimate_mi_sklearn(df:pd.DataFrame, cols:list[str], target:str, k=3, random_state:int=42):
    mi_estimates = mutual_info_classif(df[cols], df[target]
                        , n_neighbors=k, random_state=random_state, discrete_features=False)

    return pl.from_records([cols, mi_estimates], schema=["feature", "estimated_mi"]).sort("estimated_mi", descending=True)

In [22]:
# The reason sklearn's impl is slow is that it did not turn on multithreading for KDtrees.
# Sklearn also did not provide an option to turn it on, despite the fact that sklearn's KDtrees
# does have this functionality. Finished in 4.4s in this run
estimate_mi_sklearn(df_pd, cols=features, target=target).limit(10)

feature,estimated_mi
str,f64
"""column_8""",0.086362
"""column_5""",0.045589
"""column_2""",0.040451
"""column_0""",0.036647
"""column_1""",0.03106
"""column_9""",0.022108
"""column_4""",0.011848
"""column_6""",0.003617
"""column_3""",0.000802


# MRMR Feature selection Strategy

In [23]:
from mrmr import mrmr_classif # This is currently the most starred MRMR Python package on github

In [24]:
# Need to wrap it so that we get apples to apples comparison
def mrmr_package(df:pd.DataFrame, target:str, k:int) -> list[str]:
    features = list(df.columns)
    features.remove(target)
    X = df[features]
    y = df[target]
    start = perf_counter()
    output = mrmr_classif(X, y, K = k)
    end = perf_counter()
    print(f"Spent {end - start:.2f}s to compute mrmr.")
    return output

In [25]:
mrmr_package(df_pd, "target", 50)

100%|██████████| 10/10 [00:05<00:00,  1.69it/s]

Spent 8.66s to compute mrmr.


['column_8',
 'column_5',
 'column_0',
 'column_4',
 'column_2',
 'column_1',
 'column_6',
 'column_3',
 'column_7',
 'column_9']

In [26]:
start = perf_counter()
res = fs.mrmr(df, target="target", k = 50, low_memory=False)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res

INFO:dsds.fs:Running fscore to determine feature relevance...


Found 10 total features to select from. Proceeding to select top 10 features.


MRMR, fscore: 100%|██████████| 10/10 [00:00<00:00, 2499.59it/s]

Output is sorted in order of selection (max relevance min redundancy).
Spent 0.02s in computing.


['column_8',
 'column_5',
 'column_0',
 'column_4',
 'column_2',
 'column_1',
 'column_6',
 'column_3',
 'column_7',
 'column_9']

In [27]:
start = perf_counter()
res = fs.mrmr(df, target="target", k = 50, low_memory=True)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res

INFO:dsds.fs:Running fscore to determine feature relevance...


Found 10 total features to select from. Proceeding to select top 10 features.


MRMR, fscore: 100%|██████████| 10/10 [00:00<00:00, 526.22it/s]

Output is sorted in order of selection (max relevance min redundancy).
Spent 0.03s in computing.


['column_8',
 'column_5',
 'column_0',
 'column_4',
 'column_2',
 'column_1',
 'column_6',
 'column_3',
 'column_7',
 'column_9']

# Power Transform

In [28]:
# Eager transform.
start = perf_counter()
res_eager = t.power_transform(df, cols=features, strategy="yeo_johnson")
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res_eager.head() 

Spent 0.21s in computing.


target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,-1.468265,0.910684,0.659146,-1.324702,-0.141175,-0.311281,-0.882718,-0.726326,0.718409,1.203376
1,2.309445,-0.859402,2.35034,-0.859986,2.434057,-0.219607,-1.034194,-1.385379,3.108028,-1.158572
1,-3.877313,0.644551,-3.047023,0.582999,-0.723917,0.749535,-1.62376,0.947762,-1.189788,2.681555
0,0.722897,-1.905984,-0.441042,1.417185,0.993813,-0.135301,0.042292,-2.022687,-0.30606,0.977417
1,0.780848,-2.212239,-0.789209,1.273603,2.958544,-0.087321,-2.239381,-2.326567,2.339286,1.739123


In [29]:
from sklearn.preprocessing import power_transform

In [30]:
# Sklearn with Pandas

start = perf_counter()
transformed = power_transform(df_pd[features], method = "yeo-johnson", standardize=False)
end = perf_counter()
df_pd[features] = transformed
print(f"Spent {end - start:.2f}s in computing.")
df_pd.head()


Spent 0.70s in computing.


,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
0,0,-1.468265,0.910684,0.659146,-1.324702,-0.141175,-0.311281,-0.882718,-0.726326,0.718409,1.203376
1,1,2.309445,-0.859402,2.350340,-0.859986,2.434057,-0.219607,-1.034194,-1.385379,3.108028,-1.158572
2,1,-3.877313,0.644551,-3.047023,0.582999,-0.723917,0.749535,-1.623760,0.947762,-1.189788,2.681555
3,0,0.722897,-1.905984,-0.441042,1.417185,0.993813,-0.135301,0.042292,-2.022687,-0.306060,0.977417
4,1,0.780848,-2.212239,-0.789209,1.273603,2.958544,-0.087321,-2.239381,-2.326567,2.339286,1.739123


# GC

In [31]:
import gc 
gc.collect()

20